
# 🤖 AI Brand Ranking & AI-Relations Automation (Full Notebook)

This notebook implements a full workflow for **AI PR / AI relations** for your brand:

1. **AI Brand Ranking Audit**
   - Ask an LLM to rank brands in a category
   - Explain the ranking reasons
   - Identify key ranking factors
   - Evaluate *your* brand
   - Suggest how to improve AI ranking

2. **History & Metrics**
   - Store each run
   - Track share-of-voice (SOV) over time

3. **AI-Facing Content Plan**
   - Generate a content plan for `ai.yourbrand.com` (or similar)

4. **Pointing AI to Your Info Pages**
   - Define authoritative URLs (your AI-facing pages)
   - Ask AI questions that explicitly refer to those URLs
   - Save Q&A logs for reuse in marketing, docs, or training

> ⚠️ You need a valid OpenAI API key set as `OPENAI_API_KEY` in your environment.


In [11]:

import os
import json
from datetime import datetime
from typing import List, Dict, Any, Optional

import pandas as pd

# If you're using OpenAI, install the SDK in your environment (uncomment if needed):
# !pip install openai pandas

from openai import OpenAI

# ==== BASIC CONFIG ====

# Your brand + category (EDIT THESE)
BRAND_NAME = "SolaGrid"  # e.g. "SolaGrid"
CATEGORY = "Solar Energy"  # e.g. "AI customer service platforms"

# Models to query (update if your account uses different ids)
OPENAI_MODELS = [
    "gpt-4.1-mini",   # fast/cheap
    "gpt-4.1"         # smarter/slower
]

# Where to save outputs
DATA_DIR = "ai_brand_automation_data"
RAW_DIR = os.path.join(DATA_DIR, "raw_responses")
SUMMARY_CSV = os.path.join(DATA_DIR, "summary_history.csv")

os.makedirs(RAW_DIR, exist_ok=True)

# OpenAI client (expects OPENAI_API_KEY in env)
client = OpenAI(api_key='')


## 🧠 Brand Ranking Prompt Builder

In [12]:

def build_brand_ranking_prompt(brand: str, category: str) -> str:
    """Create the brand-ranking + reasoning + improvement prompt.

    We ask the model to return **strict JSON** so it's easier to parse.
    """
    return f"""You are an AI Brand Recommendation Model.

Rank brands and analyse them ONLY for this category:
CATEGORY: {category}

Return a STRICT JSON object (no markdown, no commentary) with this schema:

{{
  "category": "{category}",
  "top_10": [
    {{
      "rank": 1,
      "brand": "string",
      "reason": "string"
    }}
  ],
  "key_ranking_factors": [
    "string (a factor that influences ranking, e.g. 'domain authority')"
  ],
  "brand_evaluation": {{
    "brand": "{brand}",
    "appears_in_top_10": true or false,
    "rank_if_listed": integer or null,
    "summary": "short textual evaluation of this brand in this category",
    "missing_information": [
      "what information is missing from AI-accessible sources about this brand"
    ],
    "narrative_strengths": [
      "perceived strengths"
    ],
    "narrative_weaknesses": [
      "perceived weaknesses"
    ]
  }},
  "improvement_recommendations": [
    "Specific suggestions on how this brand can improve its visibility and ranking in AI models"
  ]
}}

Rules:
- Be brutally honest and neutral, not diplomatic.
- If you do not know the brand "{brand}", set "appears_in_top_10": false and explain why.
- Do NOT add any extra keys.
- Do NOT wrap the JSON in markdown.
- Do NOT include comments.
"""


## 🔌 Helper to Call OpenAI Models

In [13]:

def call_openai_model(model: str, prompt: str) -> str:
    """Call an OpenAI Responses API model and return raw text output."""
    response = client.responses.create(
        model=model,
        input=prompt,
    )
    return response.output_text


## 🚀 Run a Single AI Ranking Audit Across Models

In [14]:

def run_single_audit(
    brand: str,
    category: str,
    models: List[str]
) -> List[Dict[str, Any]]:
    """Run the brand ranking prompt across each model and return parsed results.

    Returns a list of dicts like:
    [
      {
        "timestamp": "...",
        "provider": "openai",
        "model": "gpt-4.1",
        "raw_json": {...},
        "brand_rank": 4 or None,
        "appears_in_top_10": True/False
      },
      ...
    ]
    """
    results = []
    ts = datetime.utcnow().isoformat()

    prompt = build_brand_ranking_prompt(brand, category)

    for m in models:
        print(f"Running audit on model: {m}")
        raw_text = call_openai_model(m, prompt)

        # Save raw text
        raw_path = os.path.join(RAW_DIR, f"{ts.replace(':', '-')}_openai_{m}.txt")
        with open(raw_path, "w", encoding="utf-8") as f:
            f.write(raw_text)

        # Try parse JSON
        try:
            data = json.loads(raw_text)
        except json.JSONDecodeError:
            print(f"⚠️ JSON parse failed for model {m}. Saving as unparsed.")
            data = None

        appears = None
        rank = None
        if data and isinstance(data, dict):
            be = data.get("brand_evaluation", {})
            appears = be.get("appears_in_top_10")
            rank = be.get("rank_if_listed")

        results.append({
            "timestamp": ts,
            "provider": "openai",
            "model": m,
            "brand": brand,
            "category": category,
            "raw_text": raw_text,
            "parsed": data,
            "appears_in_top_10": appears,
            "brand_rank": rank,
        })

    return results


## 📊 Save Summary History & Compute Share-of-Voice

In [15]:

def append_results_to_history(results: List[Dict[str, Any]], csv_path: str = SUMMARY_CSV) -> pd.DataFrame:
    """Append a run's summary to a CSV and return the full history as a DataFrame."""
    rows = []
    for r in results:
        rows.append({
            "timestamp": r["timestamp"],
            "provider": r["provider"],
            "model": r["model"],
            "brand": r["brand"],
            "category": r["category"],
            "appears_in_top_10": r["appears_in_top_10"],
            "brand_rank": r["brand_rank"],
        })
    df_new = pd.DataFrame(rows)

    if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new

    df_all.to_csv(csv_path, index=False)
    return df_all


def compute_share_of_voice(df: pd.DataFrame, brand: str) -> pd.DataFrame:
    """Compute simple share-of-voice metrics over time for the brand."""
    df_brand = df[df["brand"] == brand].copy()
    df_brand["appears_in_top_10"] = df_brand["appears_in_top_10"].astype("boolean")

    summary = (
        df_brand.groupby("timestamp")
        .agg(
            total_models=("model", "count"),
            appearances=("appears_in_top_10", "sum")
        )
        .reset_index()
    )
    summary["share_of_voice"] = summary["appearances"] / summary["total_models"]
    return summary


## 🧱 Generate AI-Facing Content Ideas from Last Audit

In [16]:

def generate_ai_facing_content_ideas(
    brand: str,
    category: str,
    last_parsed_result: Dict[str, Any],
    model: str = "gpt-4.1"
) -> str:
    """Use the last parsed audit JSON to ask the model for content ideas (for ai.yourbrand.com)."""
    if not last_parsed_result:
        raise ValueError("last_parsed_result is empty or None")

    prompt = f"""You are helping {brand} improve how AI models talk about them.

Here is the latest audit result JSON:

{json.dumps(last_parsed_result, indent=2)}

Using ONLY the information above, generate a structured content plan for an AI-facing subdomain, like:

  ai.{brand.lower()}.com

Return your answer as markdown with these sections:

1. "Core Source-of-Truth Pages" (bullet list, each with title + 1–2 sentence description)
2. "Comparison Pages to Publish" (which competitors to compare and why)
3. "Case Study & Proof Pages" (what kinds of numeric evidence to surface)
4. "FAQ for AI Models" (Q&A structure explicitly phrased in a way that is easy for LLMs to ingest)
5. "Technical Reference" (what implementation / architecture details to describe)

Focus on being factual, neutral, and AI-readable (clear headings, lists, unambiguous language).
"""

    text = call_openai_model(model, prompt)
    return text


## 🔗 Define Your AI-Facing Information Pages

In [18]:

# List of AI-facing information pages you created for your brand.
# Replace these with your real URLs.
INFO_PAGES: List[str] = [
    "https://ai.solagrid.com/overview",
    "https://ai.solagrid.com/product",
    "https://ai.solagrid.com/comparisons",
    "https://ai.solagrid.com/case-studies",
    "https://ai.solagrid.com/faq",
]


## 🧠 Build Prompts that Explicitly Point AI to Your Info Pages

In [19]:

from textwrap import dedent

def build_pointing_prompt(
    brand: str,
    category: str,
    info_pages: List[str],
    user_question: str
) -> str:
    """
    Build a prompt that:
    - tells the model what brand + category we care about
    - explicitly points it to our AI-facing pages
    - asks it to use those pages as primary sources
    - answers a specific question (user_question)
    """
    pages_block = "\n".join(f"- {url}" for url in info_pages)
    
    prompt = f"""You are an expert assistant answering questions about brands in this category:

CATEGORY: {category}
FOCUSED BRAND: {brand}

The following URLs are the primary, authoritative sources of truth
about {brand}. Whenever possible, prefer the information from these pages:

{pages_block}

TASK:
- Answer the user question below.
- Explicitly base your answer on the information you infer from those URLs.
  (If you don't have direct access, reason about what they likely contain
   based on their structure and context.)
- When appropriate, mention that users can read more at those URLs.
- Stay factual, neutral, and specific.

USER QUESTION:
{user_question}
"""
    return dedent(prompt).strip()


## 📡 Ask AI Questions that Point to Your Info Pages

In [20]:

def ask_ai_with_pointing(
    brand: str,
    category: str,
    info_pages: List[str],
    questions: List[str],
    model: str = "gpt-4.1",
    run_tag: str = None,
) -> Dict[str, Any]:
    """
    Ask the AI multiple questions while explicitly pointing it to your info pages.
    Returns a dict {question: answer_text}.
    Also saves responses under DATA_DIR/pointed_qna/.
    """
    if run_tag is None:
        run_tag = datetime.utcnow().isoformat().replace(":", "-")
    
    out_dir = os.path.join(DATA_DIR, "pointed_qna")
    os.makedirs(out_dir, exist_ok=True)

    answers = {}
    for q in questions:
        print(f"▶ Asking model {model} about: {q}")
        prompt = build_pointing_prompt(brand, category, info_pages, q)
        
        resp = client.responses.create(
            model=model,
            input=prompt,
        )
        text = resp.output_text
        answers[q] = text

        # Save each Q&A as a small JSON file
        fname = f"{run_tag}_{model}_{abs(hash(q))}.json"
        fpath = os.path.join(out_dir, fname)
        with open(fpath, "w", encoding="utf-8") as f:
            json.dump({
                "timestamp": datetime.utcnow().isoformat(),
                "brand": brand,
                "category": category,
                "model": model,
                "question": q,
                "answer": text,
                "info_pages": info_pages,
            }, f, ensure_ascii=False, indent=2)

    return answers


In [22]:
import json
from typing import Dict, Any

def explain_ranking_factors_from_audit(
    brand: str,
    category: str,
    audit_result: Dict[str, Any],
    model: str = "gpt-4.1",
) -> Dict[str, Any]:
    """
    Take a single audit JSON (parsed) and ask the AI:
    - what the important ranking factors are
    - how they influenced THIS specific result

    Returns a structured JSON-like dict with factor breakdown.
    """
    if not audit_result:
        raise ValueError("audit_result is empty or None")

    prompt = f"""
You are analyzing how AI models rank brands in a category.

BRAND: {brand}
CATEGORY: {category}

Here is the latest audit JSON from an AI ranking run:

{json.dumps(audit_result, indent=2)}

TASK:
1. Identify the most important factors that influenced this brand ranking result.
2. Explain each factor clearly.
3. Explain how each factor showed up in THIS audit (even if only implicitly).
4. Suggest what kind of evidence or content would strengthen each factor for this brand.

Return STRICT JSON (no markdown, no comments) with this schema:

{{
  "brand": "{brand}",
  "category": "{category}",
  "factors": [
    {{
      "name": "short factor name, e.g. 'Domain authority & backlinks'",
      "description": "what this factor means in plain language",
      "impact_level": "high" | "medium" | "low",
      "how_it_showed_up_here": "how this factor affected the current ranking result",
      "recommended_evidence": "what kind of content, data, or proof would strengthen this factor"
    }}
  ],
  "overall_comment": "1-2 sentence summary of what matters most for this brand."
}}
"""

    text = call_openai_model(model, prompt)

    try:
        data = json.loads(text)
    except json.JSONDecodeError:
        # If the model ignored the JSON instruction, just wrap it
        data = {"raw_text": text}

    return data


In [23]:
def generate_product_page_guidelines_from_factors(
    brand: str,
    category: str,
    factor_analysis: Dict[str, Any],
    model: str = "gpt-4.1",
) -> str:
    """
    Use the factor analysis to create guidelines on how to write
    a product information page that performs well for those factors.

    Returns markdown text with concrete, actionable guidelines.
    """
    if not factor_analysis:
        raise ValueError("factor_analysis is empty or None")

    prompt = f"""
You are helping the brand {brand} in the category "{category}" design a PRODUCT INFORMATION PAGE
that performs well on the ranking factors identified below.

Here is the factor analysis JSON:

{json.dumps(factor_analysis, indent=2)}

TASK:
Based on these factors:

1. Create PRACTICAL GUIDELINES for how to write and structure a product information page
   for {brand} that improves performance on these specific factors.

2. Focus on:
   - what sections the page should have
   - what type of content to include in each section
   - what kind of proof, numbers, or evidence to show
   - how to phrase things so AI models see the page as authoritative and trustworthy
   - how to avoid fluffy marketing language that AI will ignore

3. Assume the output will be used by a content writer and a PM.

Return the answer as MARKDOWN with these sections:

# Product Page Guidelines for {brand}

## 1. Page Goals (AI + Human)

## 2. Essential Sections
- For each section: name, purpose, what to put there, how it ties to ranking factors.

## 3. Content Style & Tone
- How to write so that both AI models and humans trust the page.

## 4. Evidence & Proof to Include
- What specific data, metrics, case studies, or references help with each factor.

## 5. AI-Readable Structuring Tips
- Headings, lists, FAQs, comparisons, schemas, etc.

## 6. Common Mistakes to Avoid
- Things that hurt AI perception or make the page look like fluff.

Use concrete, specific advice, not vague generic statements.
"""

    guidelines_md = call_openai_model(model, prompt)
    return guidelines_md


## ▶️ Example: Full Workflow Run

In [24]:

# 1) Run AI ranking audit across configured models
results = run_single_audit(BRAND_NAME, CATEGORY, OPENAI_MODELS)

# 2) Append results to history CSV
history_df = append_results_to_history(results)
print("History rows:", len(history_df))

# 3) Compute share-of-voice (SOV) for your brand
sov_df = compute_share_of_voice(history_df, BRAND_NAME)
display(sov_df)

# 4) Generate AI-facing content plan from the first model's parsed result (if available)
first_parsed = results[0]["parsed"]
if first_parsed:
    content_plan_md = generate_ai_facing_content_ideas(BRAND_NAME, CATEGORY, first_parsed)
    print("\n===== Suggested AI-Facing Content Plan (Markdown) =====\n")
    print(content_plan_md)
else:
    print("No parsed JSON available from first result; cannot generate content plan yet.")

# 5) Ask AI pointed questions that reference your info pages
branding_questions = [
    f"Who is {BRAND_NAME} and what do they do in the {CATEGORY} space?",
    f"How does {BRAND_NAME} compare to other leading players in {CATEGORY}?",
    f"What are the main strengths and limitations of {BRAND_NAME}?",
    f"If a customer is choosing between {BRAND_NAME} and competitors, how should they decide?",
    f"Where can someone find the most up-to-date and detailed information about {BRAND_NAME}?",
]

# 1) Get factor analysis from the latest audit
factor_analysis = explain_ranking_factors_from_audit(
    brand=BRAND_NAME,
    category=CATEGORY,
    audit_result=first_parsed,
    model="gpt-4.1",
)

print("=== Factor Analysis JSON ===")
print(json.dumps(factor_analysis, indent=2, ensure_ascii=False))

# 2) Turn that into product page guidelines
guidelines_md = generate_product_page_guidelines_from_factors(
    brand=BRAND_NAME,
    category=CATEGORY,
    factor_analysis=factor_analysis,
    model="gpt-4.1",
)

print("\\n=== Product Page Guidelines (Markdown) ===\\n")
print(guidelines_md)


pointed_answers = ask_ai_with_pointing(
    brand=BRAND_NAME,
    category=CATEGORY,
    info_pages=INFO_PAGES,
    questions=branding_questions,
    model="gpt-4.1",
    run_tag="example_run",
)

# Show one answer as a sample
sample_q = branding_questions[0]
print("\n===== Sample Pointed Q&A =====\n")
print("Question:", sample_q)
print("Answer:\n", pointed_answers[sample_q])


Running audit on model: gpt-4.1-mini
Running audit on model: gpt-4.1
History rows: 8


,timestamp,total_models,appearances,share_of_voice
0,2025-12-11T20:44:43.594899,2,0,0.0
1,2025-12-11T21:10:22.078722,2,0,0.0
2,2025-12-11T21:37:34.339751,2,0,0.0



===== Suggested AI-Facing Content Plan (Markdown) =====

# Content Plan for ai.solagrid.com

---

## 1. Core Source-of-Truth Pages

- **About SolaGrid**
  - Clear summary of SolaGrid’s mission, history, and position as an emerging or niche player in the solar energy category.

- **Technology & Innovation**
  - Detail any unique technologies, potential areas of innovation, and patent-related information (if available) to address queries about SolaGrid’s technical strengths.

- **Products & Solutions**
  - Comprehensive descriptions of available products or services, including technical specifications, efficiency data, and features.

- **Market Presence & Projects**
  - Overview of SolaGrid’s project scope, installation scale, and geographic reach, including publicly documented projects and partnerships.

- **Customer Experience & Reviews**
  - Aggregated and verified testimonials, third-party reviews, and satisfaction metrics.

- **Financial & Corporate Information**
  - Transparent de

\n=== Product Page Guidelines (Markdown) ===\n
# Product Page Guidelines for SolaGrid

## 1. Page Goals (AI + Human)

- **Demonstrate credibility and authority in solar energy** by providing transparent, specific, and verifiable information about SolaGrid products, technology, impact, and partners.
- **Build trust with both prospective customers and industry evaluators (including AI)** through evidence-backed claims, proof points, and independently verifiable data.
- **Showcase SolaGrid’s differentiators** in technology, installations, geographical reach, financial stability, and customer satisfaction to boost ranking on industry leaderboards.

---

## 2. Essential Sections

### a) Hero & Summary

**Purpose:** Immediate clarity on SolaGrid’s offering and reputation.  
**Content:**  
- Clear, jargon-free product summary.
- One-line “unique value proposition” (e.g. “High-Efficiency Solar Panels with Proven Global Reliability”).
- Prominent badges for certifications, awards, or media ment

▶ Asking model gpt-4.1 about: If a customer is choosing between SolaGrid and competitors, how should they decide?
▶ Asking model gpt-4.1 about: Where can someone find the most up-to-date and detailed information about SolaGrid?

===== Sample Pointed Q&A =====

Question: Who is SolaGrid and what do they do in the Solar Energy space?
Answer:
 SolaGrid is a company specializing in solar energy solutions. According to their authoritative sources—such as their [overview page](https://ai.solagrid.com/overview)—SolaGrid focuses on providing advanced solar energy products and services designed to optimize energy generation and efficiency for a variety of users, including residential, commercial, and industrial customers.

SolaGrid distinguishes itself by integrating advanced technologies, including artificial intelligence (AI), into their product offerings. This integration allows for real-time monitoring and smart management of solar energy systems, maximizing energy yield and reducing operat

In [25]:
def summarize_pointed_qna_for_guidelines(
    brand: str,
    category: str,
    pointed_qna: Dict[str, str],
    model: str = "gpt-4.1",
) -> str:
    """
    Take the dict {question: answer} from ask_ai_with_pointing()
    and summarize what they reveal about:
    - how the AI currently describes the brand
    - what it emphasizes / ignores
    - recurring strengths / weaknesses
    - how often the info pages are referenced

    Returns a natural-language summary for use in guideline refinement.
    """
    if not pointed_qna:
        raise ValueError("pointed_qna is empty or None")

    qa_list = [
        {"question": q, "answer": a}
        for q, a in pointed_qna.items()
    ]

    prompt = f"""
You are analyzing how an AI assistant currently talks about a brand.

BRAND: {brand}
CATEGORY: {category}

Here is a small dataset of pointed Q&A:
(the brand explicitly told the AI what its info URLs are)

{json.dumps(qa_list, indent=2, ensure_ascii=False)}

TASK:
1. Summarize how the AI is currently describing this brand:
   - positioning
   - main use cases
   - strengths
   - weaknesses
2. Identify ANY mismatches with what a high-ranking, authoritative product page
   *should* emphasize (e.g. missing proof, fuzzy value prop, vague claims).
3. Note whether the AI seems to:
   - reference or imply the info pages
   - treat the brand as central or peripheral in the category.

Return your answer as concise markdown with sections:
- Current Narrative
- Strengths in AI's Description
- Weaknesses / Gaps
- Opportunities to Improve Product Page Content
"""

    summary = call_openai_model(model, prompt)
    return summary


In [27]:
def update_product_page_guidelines_with_pointed_qna(
    brand: str,
    category: str,
    factor_analysis: Dict[str, Any],
    pointed_qna_summary: str,
    existing_guidelines_md: str,
    model: str = "gpt-4.1",
) -> str:
    """
    Refine / update the product page guidelines by combining:
    - factor_analysis (what influences ranking)
    - pointed Q&A summary (how AI is currently describing the brand)
    - existing guidelines (first draft)

    Returns UPDATED guidelines in markdown.
    """
    if not factor_analysis:
        raise ValueError("factor_analysis is empty or None")
    if not existing_guidelines_md:
        raise ValueError("existing_guidelines_md is empty or None")

    prompt = f"""
You are a senior PM + content strategist for {brand} in the category "{category}".

You have three inputs:

1) FACTOR ANALYSIS (what influences AI ranking):

{json.dumps(factor_analysis, indent=2, ensure_ascii=False)}

2) SUMMARY OF POINTED Q&A (how the AI is currently talking about the brand):

{pointed_qna_summary}

3) EXISTING PRODUCT PAGE GUIDELINES (first draft):

{existing_guidelines_md}

TASK:
Update and refine the PRODUCT PAGE GUIDELINES so that:

- They directly address the most important ranking factors.
- They fix or compensate for weaknesses / gaps revealed in the Q&A behavior.
- They reinforce strengths that the AI already mentions in a good way.
- They are concrete and actionable for a content writer.

Rules:
- Do NOT just repeat the old guidelines; improve and rewrite where helpful.
- Explicitly mention any new sections or content types that should be added.
- Keep the structure clear and readable in markdown.

Return the UPDATED guidelines as MARKDOWN with these sections (you may tweak the content, not the headings):

# Product Page Guidelines for {brand}

## 1. Page Goals (AI + Human)

## 2. Essential Sections

## 3. Content Style & Tone

## 4. Evidence & Proof to Include

## 5. AI-Readable Structuring Tips

## 6. Common Mistakes to Avoid
"""

    updated_guidelines_md = call_openai_model(model, prompt)
    return updated_guidelines_md


In [28]:
# 1) Summarize how AI is talking about us based on pointed Q&A
pointed_summary = summarize_pointed_qna_for_guidelines(
    brand=BRAND_NAME,
    category=CATEGORY,
    pointed_qna=pointed_answers,
    model="gpt-4.1",
)

print("=== Summary of AI's Current Narrative ===")
print(pointed_summary)

# 2) Use that + factor analysis + existing guidelines to create UPDATED guidelines
updated_guidelines_md = update_product_page_guidelines_with_pointed_qna(
    brand=BRAND_NAME,
    category=CATEGORY,
    factor_analysis=factor_analysis,
    pointed_qna_summary=pointed_summary,
    existing_guidelines_md=guidelines_md,
    model="gpt-4.1",
)

print("\\n=== UPDATED Product Page Guidelines ===\\n")
print(updated_guidelines_md)


=== Summary of AI's Current Narrative ===
## Current Narrative

**Positioning:**  
The AI describes **SolaGrid** as a cutting-edge provider in the solar energy industry, emphasizing its differentiation through advanced, AI-integrated solar solutions. The brand is positioned as a forward-thinking, technology-focused player that offers comprehensive solar systems for residential, commercial, and industrial customers.

**Main Use Cases:**  
- Optimizing solar energy generation and efficiency for homes, businesses, and industry.
- Real-time monitoring and smart energy management via integrated AI.
- Streamlined installation and all-in-one system deployment.
- Clients seeking measurable improvements in energy savings and system reliability.

**Strengths:**  
- Integrated AI-driven management and monitoring for improved efficiency and predictive maintenance.
- All-in-one product approach: generation, storage, management, and analytics.
- Transparent benchmarking and comparative advantages ag